In [125]:
%%capture
!pip install stanza

In [126]:
%%capture
import stanza

stanza.download('es')
nlp = stanza.Pipeline('es')

In [127]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  p = {"src":l.split(";")[0],"target":l.split(";")[1]}
  dic.append(p)

In [132]:
import re
import string
def remove_punctuation (s):
  return re.sub('[%s]' % re.escape(string.punctuation), ' ', s)

In [133]:
def judeo_parse(s):
  if s.find("h") != -1:
    indices = [i for i, x in enumerate(s) if x == "h"]
    flag_h = 0
    for h in indices:
      string_list = list(s)
      if h != 0:
        if s[h-2] != "c":
          string_list[h] = ""
      else:
        string_list[0] = ""
    s = "".join(string_list)
  if s.find("y") != -1:
    if len(s) == 1:
      s = s.replace("y","i")
  if s.find("ca") != -1:
    s = s.replace("ca","ka")
  if s.find("co") != -1:
    s = s.replace("co","ko")
  if s.find("ñ") != -1:
    s = s.replace("ñ","ny")
  if s.find("qu") != -1:
    s = s.replace("qu","k")
  if s.find("rd") != -1:
    s = s.replace("rd","dr")
  if s.find("cr") != -1:
    s = s.replace("cr","kr")
  if s.find("ll") != -1:
    s = s.replace("ll","y")
  if s.find("g") != -1:
    s = s.replace("g","j")
  return s

In [134]:
def complete_word(mayor,menor):
  for l in range(len(mayor)-len(menor)):
    menor = menor+" "
  return menor

def get_suffix(a,b):
  flag = 0
  mayor = ""
  menor = ""

  if len(a) > flag:
    mayor = a
    menor = b
    flag = len(a)
  if len(b) > flag:
    mayor = b
    menor = a
  menor = complete_word(mayor,menor)

  f = []
  for x, y in zip(mayor,menor):
    if x == y:
      f.append(x)

  h = ''.join([str(elem) for elem in f])

  for l in range(len(a)-len(h)):
    h = h+" "

  p = []
  for x, y in zip(a,h):
    if x != y:
      p.append(x)

  p = ''.join([str(elem) for elem in p])
  
  return p

In [169]:
def conj_verb(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  verbo_inf = verbo_esp.lemma
  if "ar" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('ar',1))
  if "er" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('er',1))
  if "ir" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
    verbo_ladino = ''.join(verbo_ladino.rsplit('ir',1))

  if "ando" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ando"
  elif "iendo" in verbo_phrase:
    verbo_ladino = verbo_ladino+"iendo"
  elif "ado" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ado"
  elif "ido" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ido"
  else:
    suff = get_suffix(verbo_phrase,verbo_inf)
    verbo_ladino = verbo_ladino+suff
  return verbo_ladino

In [179]:
def conj_adj_sustantivo(word_esp,word_ladino):
  if "Sing" in word_esp.feats:
    suff_word_esp = word_esp.text[len(word_esp.text)-1:]
    root_word_lad = word_ladino[:len(word_ladino)-1]
    word_ladino = root_word_lad+suff_word_esp
    return word_ladino
  if "Plur" in word_esp.feats:
    suff_word_esp = word_esp.text[len(word_esp.text)-2:]
    root_word_lad = word_ladino[:len(word_ladino)-1]
    word_ladino = root_word_lad+suff_word_esp
    return word_ladino

In [183]:
phrase="Hoy me compre dos casas"

In [184]:
def translate(phrase):
  doc = nlp(phrase)
  jud_phrase = ""
  w = ""
  for sent in doc.sentences:
    for word in sent.words:
      flag1 = 0
      español_word = word.text
      español_word = español_word.lower()
      español_word = remove_punctuation(español_word)
      for d in dic:
        if word.lemma == d["src"] or word.text == d["src"]:
          ladino_word = d["target"].replace(" ","").replace("\n","")
          if word.lemma == ladino_word or word.text == ladino_word:
            w = español_word
            break
          if word.upos == "VERB":
            w = conj_verb(word,ladino_word)
          elif word.upos == "NOUN" or word.upos == "ADJ":
            w = conj_adj_sustantivo(word,ladino_word)
          else:
            w = ladino_word
          flag1 = 1
          break
      if flag1 == 0:
          w = judeo_parse(español_word)
      jud_phrase += w.replace("\n","") + " "
  return jud_phrase.strip()

In [185]:
translate(phrase)

'oy me marke dos kazas'

Testing

In [124]:
import pandas as pd

df = pd.read_excel("una-fraze-al-dia_lad-tur-eng-spa.xlsx")

esp=[]
lad_correct = []
lad_trans = []

for a in df.index:
  esp.append(df["Español"][a])
  lad_correct.append(df["Ladino"][a])
  lad_trans.append(translate(df["Español"][a]))

p = {'Español':esp,'Ladino':lad_correct,'Ladino_Trans':lad_trans}

df_1 = pd.DataFrame(p)
df_1.head()

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


TypeError: ignored

In [113]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  word_esp = l.split(";")[0]
  word_ladino = l.split(";")[1]
  if len(word_esp.split(" ")) == 1 or len(word_esp.split(",")) == 1: 
    doc = nlp(word_esp)
    for sent in doc.sentences:
      for word in sent.words:
        if word.upos == "NOUN" and str(word.feats) != "None":
          if "Sing" in word.feats and "Masc" in word.feats:
            if word_esp[len(word_esp)-1:] == "o":
              root_word_esp = word_esp[:len(word_esp)-1]
              root_word_lad = word_ladino[:len(word_ladino)-2]
              word_esp = root_word_esp+"a"
              word_ladino = root_word_lad+"a"
              p = word_esp+";"+word_ladino
              dic.append(p)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


Using a new POS-Tagging for Spanish

In [ ]:
!pip install nltk

In [ ]:
import nltk
tagged_sentences = nltk.corpus.conll2002.tagged_sents('esp.train')

In [ ]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:3],
        'prefix-3': sentence[index][:4],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-3:],
        'suffix-3': sentence[index][-4:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [ ]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [ ]:
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]

print(len(training_sentences))
print(len(test_sentences))

def transform_to_dataset(tagged_sentences):
    X, y = [], []
    
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged),index))
            y.append(tagged[index][1])
    return X, y

X, y = transform_to_dataset(training_sentences)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])

clf.fit(X[:20000], y[:20000])

print('Training completed')

In [ ]:
def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return list(zip(sentence, tags))

In [ ]:
from nltk import word_tokenize
print (pos_tag(word_tokenize('El canciller de Alemania llegó el lunes a Rusia')))